In [1]:
import keras
import numpy as np
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')


Using TensorFlow backend.


606208/600901 [==============================] - 2s 3us/step


In [2]:
text = open(path).read().lower()

In [3]:
print('Corpus length', len(text))

Corpus length 600901


In [6]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('unique characters: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) 
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t,char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
unique characters:  59
Vectorization...


In [9]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [10]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [14]:
import random
import sys
for epoch in range(1,2):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index+maxlen]
    print('--- Generating with seed: "'+ generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
200281/200281 [==============================] - 848s 4ms/step - loss: 1.8876
--- Generating with seed: "nged that the worst of all tastes, the taste for
the uncondi"
----- temperature: 0.2
nged that the worst of all tastes, the taste for
the uncondicience and serves and sense and says and instinct they has a demant to seem and self and in the states and in the intermanders of the states and something in a demant to self and and the stapsical interman and interman and serves and interman and in the self and interman instinct, and interman and in the self and in the sense and self and in the present and the self and sense and and in the stande----- temperature: 0.5
 in the present and the self and sense and and in the standers of the have in order and the sense of the
entarination of the lays and a thereself to and in the will on the world on the haster and designtely and its insciously, and is a person and invensely and nature, as wholaruse to are and emplicical who h